In [35]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import time

# Setting pandas to display columns
pd.set_option('display.max_columns', None)

In [36]:
nfl_small2_end_of_drive = pd.read_csv('nfl_small_end_of_drive.csv',index_col=1,\
    dtype= {'ARI' : 'str','ATL' : 'str', 'BAL' : 'str', 'BUF' : 'str', 'CAR' : 'str',
             'CHI' : 'str', 'CIN' : 'str', 'CLE' : 'str', 'DAL' : 'str', 'DEN' : 'str',
             'DET' : 'str', 'GB' : 'str', 'HOU' : 'str', 'IND' : 'str', 'JAX' : 'str',
             'KC' : 'str', 'LA' : 'str', 'LAC' : 'str', 'MIA' : 'str', 'MIN' : 'str',
             'NE' : 'str', 'NO' : 'str', 'NYG' : 'str','NYJ' : 'str', 'OAK' : 'str',
             'PHI' : 'str', 'PIT' : 'str','SEA' : 'str', 'SF' : 'str', 'TB' : 'str',
             'TEN' : 'str', 'WAS' : 'str'})

In [37]:
# Creating Home and Away Win Columns
nfl_small2_end_of_drive['home_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              > nfl_small2_end_of_drive['total_away_score'],1,0)

nfl_small2_end_of_drive['away_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              < nfl_small2_end_of_drive['total_away_score'],1,0)

In [38]:
# Getting teams to add team matrix for easier filtering
teams = list(nfl_small2_end_of_drive.groupby('posteam').sum().index)
teams[:4]

['ARI', 'ATL', 'BAL', 'BUF']

In [66]:
nfl_small2_end_of_drive.play_type.unique()

array(['kickoff', 'run', 'pass', 'no_play', 'field_goal', 'punt',
       'extra_point', 'qb_kneel', 'qb_spike'], dtype=object)

In [67]:
nfl_small2_end_of_drive[nfl_small2_end_of_drive['play_type']=='field_goal']

,index,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,week,home_team_win,away_team_win
play_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
285,270419,2015091000,2015-09-10 00:00:00,09:44,584.0,1484.0,3284.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,26.0,1,0,4.0,0.0,NE 26,12,54,"(9:44) J.Scobee 44 yard field goal is No Good,...",field_goal,0.0,0,0.0,0.0,missed,44.0,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:15:48.000000000,2015-09-10 20:55:48,2015-09-10 20:55:48,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.30,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1,0,0
1208,270458,2015091000,2015-09-10 00:00:00,08:43,523.0,523.0,2323.0,Half1,0,2,NE,PIT,PIT,away,NE,NE,28.0,5,0,4.0,0.0,NE 28,3,52,"(8:43) J.Scobee 46 yard field goal is No Good,...",field_goal,0.0,0,0.0,0.0,missed,46.0,NaN,NaN,NaN,7,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 01:03:51.000000000,2015-09-10 21:43:51,2015-09-10 21:43:51,2015-09-10 22:00:00,1010.1,Foxborough,16.7,FieldTurf CORE,87.0,0.15,Open,NE,18.9,2015-09-10 22:00:00,2015-09-11 02:00:00,NaN,30.0,24.1,0,PIT @ NE 2015-09-10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1,1,0
1852,270486,2015091000,2015-09-10 00:00:00,00:07,7.0,7.0,1807.0,Half1,0,2,NE,PIT,PIT,away,NE,NE,26.0,7,1,4.0,0.0,NE 26,5,57,"(:07) J.Scobee 44 yard field goal is GOOD, Cen...",field_goal,0.0,0,0.0,0.0,made,44.0,NaN,NaN,NaN,14,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 01:29:39.000000000,2015-09-10 22:09:39,2015-09-10 22:09:39,2015-09-10 22:00:00,1010.1,Foxborough,16.7,FieldTurf CORE,87.0,0.15,Open,NE,18.9,2015-09-10 22:00:00,2015-09-11 02:00:00,NaN,30.0,24.1,0,PIT @ NE 2015-09-10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1,1,0
2879,270533,2015091000,2015-09-10 00:00:00,11:42,702.0,702.0,702.0,Half2,0,4,NE,PIT,PIT,away,NE,NE,6.0,12,1,4.0,1.0,NE 6,6,67,"(11:42) J.Scobee 24 yard field goal is GOOD, C...",field_goal,0.0,0,0.0,0.0,made,24.0,NaN,NaN,NaN,21,14,11.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 02:24:54.000000000,2015-09-10 23:04:54,2015-09-10 23:04:54,2015-09-10 23:00:00,1009.2,Foxborough,16.6,FieldTurf CORE,90.0,0.00,Open,NE,18.3,2015-09-10 23:00:00,2015-09-11 03:00:00,NaN,20.0,29.5,0,PIT @ NE 2015-09-10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1,1,0
299,270595,2015091300,2015-09-13 00:00:00,09:32,572.0,1472.0,3272.0,Half1,0,1,CHI,GB,CHI,home,GB,GB,10.0,1,1,4.0,0.0,GB 10,8,70,(9:32) (Field Goal formation) R.Gould 28 yard ...,field_goal,0.0,0,0.0,0.0,made,28.0,NaN,NaN,NaN,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2015-09-13 13:00:00,2015.0,0 days 00:16:24.000000000,2015-09-13 13:16:24,2015-09-13 13:16:24,2015-09-13 13:00:00,10

In [39]:
nfl_small2_end_of_drive.head(2)

,index,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,week,home_team_win,away_team_win
play_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36,270409,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,35.0,1,0,NaN,0.0,NE 35,0,0,S.Gostkowski kicks 65 yards from NE 35 to end ...,kickoff,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1,0,0
51,270410,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,80.0,1,0,1.0,0.0,PIT 20,10,18,(15:00) De.Williams right tackle to PIT 38 for...,run,18.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1,0,0


In [90]:
# Using a form of Joey's agg function to get to end of game stats
def my_aggfunc(x):
    values = {
        'pass_home':x[x['posteam_type']=='home']['play_type'].eq('pass').sum(),
        'pass_away':x[x['posteam_type']=='away']['play_type'].eq('pass').sum(),
        
        'run_home':x[x['posteam_type']=='home']['play_type'].eq('run').sum(),
        'run_away':x[x['posteam_type']=='away']['play_type'].eq('run').sum(),
        
        'TO_home':x[x['posteam_type']=='home']['INT'].sum() +\
        x[x['posteam_type']=='home']['fumble'].sum(),
        'TO_away':x[x['posteam_type']=='away']['INT'].sum() +\
        x[x['posteam_type']=='away']['fumble'].sum(),
        
        'yards_home':x[(x['posteam_type']=='home') & (x['end_of_drive']==1)]['ydsnet'].sum(),
        'yards_away':x[(x['posteam_type']=='away') & (x['end_of_drive']==1)]['ydsnet'].sum(),
        
        'fg_at_home':x[(x['posteam_type']=='home')&(x['play_type']=='field_goal')]['play_type'].count(),
        'fg_at_away':x[(x['posteam_type']=='away')&(x['play_type']=='field_goal')]['play_type'].count(),
        'avg_kick_dist':x[(x['posteam_type']=='away')&(x['play_type']=='field_goal')]['kick_distance'].mean(),
        
        'fg_md_home':x[(x['posteam_type']=='home')&(x['play_type']=='field_goal')&(x['field_goal_result']=='made')]['play_type'].count(),
        'fg_md_away':x[(x['posteam_type']=='away')&(x['play_type']=='field_goal')&(x['field_goal_result']=='made')]['play_type'].count(),
        
        
        'pass_yards_home':x[(x['posteam_type']=='home')&(x['play_type']=='pass')]['yards_gained'].sum(),
        'pass_yards_away':x[(x['posteam_type']=='away')&(x['play_type']=='pass')]['yards_gained'].sum(),
        
        'run_yards_home':x[(x['posteam_type']=='home')&(x['play_type']=='run')]['yards_gained'].sum(),
        'run_yards_away':x[(x['posteam_type']=='away')&(x['play_type']=='run')]['yards_gained'].sum()
        
    }
    return pd.Series(values)

In [91]:
# Initializing end of game df
new_df = pd.DataFrame()
new_df = nfl_small2_end_of_drive.groupby(['game_id']).apply(my_aggfunc)

In [92]:
# Setting teams and scores
new_df['home_team'] = nfl_small2_end_of_drive.groupby('game_id')['home_team'].first()
new_df['away_team'] = nfl_small2_end_of_drive.groupby('game_id')['away_team'].first()
new_df['home_score'] = nfl_small2_end_of_drive.groupby('game_id')['total_home_score'].last()
new_df['away_score'] = nfl_small2_end_of_drive.groupby('game_id')['total_away_score'].last()

In [93]:
# Creating DF with rows for each team in every game
test_df = new_df.reset_index()[['game_id','home_team','away_team']]\
                .melt(id_vars='game_id',var_name = 'home_away',value_name="Team")
test_df

,game_id,home_away,Team
0,2015091000,home_team,NE
1,2015091300,home_team,CHI
2,2015091301,home_team,LA
3,2015091302,home_team,JAX
4,2015091303,home_team,WAS
...,...,...,...
1977,2018121608,away_team,TEN
1978,2018121609,away_team,SEA
1979,2018121610,away_team,NE
1980,2018121611,away_team,PHI


In [94]:
# Extracting year to be able to aggregate wins by year
test_df['year'] = test_df['game_id'].astype('str').str.extract(r'(^[0-9]{4})')
test_df.head()

,game_id,home_away,Team,year
0,2015091000,home_team,NE,2015
1,2015091300,home_team,CHI,2015
2,2015091301,home_team,LA,2015
3,2015091302,home_team,JAX,2015
4,2015091303,home_team,WAS,2015


In [95]:
# Merging with team home stats
test_df_home = test_df.merge(new_df.reset_index(),how='inner',left_on=['game_id','Team'],\
                             right_on=['game_id','home_team'])

In [96]:
# Setting variables for Home Team
test_df_home['team_against'] = test_df_home[test_df_home['home_away']=='home_team']['away_team']
test_df_home['score'] = test_df_home[test_df_home['home_away']=='home_team']['home_score']
test_df_home['score_against'] = test_df_home[test_df_home['home_away']=='home_team']['away_score']

test_df_home['yards'] = test_df_home[test_df_home['home_away']=='home_team']['yards_home']
test_df_home['yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['yards_away']

test_df_home['run_plays'] = test_df_home[test_df_home['home_away']=='home_team']['run_home']
test_df_home['run_plays_against'] = test_df_home[test_df_home['home_away']=='home_team']['run_away']

test_df_home['pass_plays'] = test_df_home[test_df_home['home_away']=='home_team']['pass_home']
test_df_home['pass_plays_against'] = test_df_home[test_df_home['home_away']=='home_team']['pass_away']

test_df_home['to_for'] = test_df_home[test_df_home['home_away']=='home_team']['TO_home']
test_df_home['to_against'] = test_df_home[test_df_home['home_away']=='home_team']['TO_away']

test_df_home['fg_at'] = test_df_home[test_df_home['home_away']=='home_team']['fg_at_home']
test_df_home['fg_md'] = test_df_home[test_df_home['home_away']=='home_team']['fg_md_home']

test_df_home['fg_at_against'] = test_df_home[test_df_home['home_away']=='home_team']['fg_at_away']
test_df_home['fg_md_against'] = test_df_home[test_df_home['home_away']=='home_team']['fg_md_away']

test_df_home['pass_yards'] = test_df_home[test_df_home['home_away']=='home_team']['pass_yards_home']
test_df_home['pass_yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['pass_yards_away']

test_df_home['run_yards'] = test_df_home[test_df_home['home_away']=='home_team']['run_yards_home']
test_df_home['run_yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['run_yards_away']

In [97]:
# Creating away only df to merge records from hoem and away team stats
test_df_away = test_df.merge(new_df.reset_index(),how='inner',left_on=['game_id','Team'],right_on=['game_id','away_team'])
test_df_away[test_df_away['game_id']==2017090700]

,game_id,home_away,Team,year,pass_home,pass_away,run_home,run_away,TO_home,TO_away,yards_home,yards_away,fg_at_home,fg_at_away,avg_kick_dist,fg_md_home,fg_md_away,pass_yards_home,pass_yards_away,run_yards_home,run_yards_away,home_team,away_team,home_score,away_score
511,2017090700,away_team,KC,2017,39.0,38.0,34.0,24.0,2.0,1.0,425.0,538.0,2.0,0.0,NaN,2.0,0.0,247.0,352.0,125.0,191.0,NE,KC,27,42


In [98]:
# Setting variables for Away Team
test_df_away['team_against'] = test_df_away[test_df_away['home_away']=='away_team']['home_team']
test_df_away['score'] = test_df_away[test_df_away['home_away']=='away_team']['away_score']
test_df_away['score_against'] = test_df_away[test_df_away['home_away']=='away_team']['home_score']

test_df_away['yards'] = test_df_away[test_df_away['home_away']=='away_team']['yards_away']
test_df_away['yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['yards_home']

test_df_away['run_plays'] = test_df_away[test_df_away['home_away']=='away_team']['run_away']
test_df_away['run_plays_against'] = test_df_away[test_df_away['home_away']=='away_team']['run_home']

test_df_away['pass_plays'] = test_df_away[test_df_away['home_away']=='away_team']['pass_away']
test_df_away['pass_plays_against'] = test_df_away[test_df_away['home_away']=='away_team']['pass_home']

test_df_away['to_for'] = test_df_away[test_df_away['home_away']=='away_team']['TO_away']
test_df_away['to_against'] = test_df_away[test_df_away['home_away']=='away_team']['TO_home']

test_df_away['fg_at'] = test_df_away[test_df_away['home_away']=='away_team']['fg_at_away']
test_df_away['fg_md'] = test_df_away[test_df_away['home_away']=='away_team']['fg_md_away']

test_df_away['fg_at_against'] = test_df_away[test_df_away['home_away']=='away_team']['fg_at_home']
test_df_away['fg_md_against'] = test_df_away[test_df_away['home_away']=='away_team']['fg_md_home']

test_df_away['pass_yards'] = test_df_away[test_df_away['home_away']=='away_team']['pass_yards_away']
test_df_away['pass_yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['pass_yards_home']

test_df_away['run_yards'] = test_df_away[test_df_away['home_away']=='away_team']['run_yards_away']
test_df_away['run_yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['run_yards_home']

test_df_away[test_df_away['game_id']==2017090700]

,game_id,home_away,Team,year,pass_home,pass_away,run_home,run_away,TO_home,TO_away,yards_home,yards_away,fg_at_home,fg_at_away,avg_kick_dist,fg_md_home,fg_md_away,pass_yards_home,pass_yards_away,run_yards_home,run_yards_away,home_team,away_team,home_score,away_score,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against
511,2017090700,away_team,KC,2017,39.0,38.0,34.0,24.0,2.0,1.0,425.0,538.0,2.0,0.0,NaN,2.0,0.0,247.0,352.0,125.0,191.0,NE,KC,27,42,NE,42,27,538.0,425.0,24.0,34.0,38.0,39.0,1.0,2.0,0.0,0.0,2.0,2.0,352.0,247.0,191.0,125.0


In [110]:
# Filtering done to columns needed in combined Home and Away dataframe
new_cols = ['game_id','Team','team_against','score','score_against','yards','yards_against','run_plays',\
            'run_plays_against','pass_plays','pass_plays_against','to_for','to_against','fg_at','avg_kick_dist',\
           'fg_md','fg_at_against','fg_md_against','pass_yards','pass_yards_against','run_yards','run_yards_against']


In [111]:
# Combining home and away dfs so each team has a record for each game
combined_df = test_df_home[new_cols].append(test_df_away[new_cols],ignore_index=True)
combined_df[combined_df['game_id']==2017090700]

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,avg_kick_dist,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against
511,2017090700,NE,KC,27,42,425.0,538.0,34.0,24.0,39.0,38.0,2.0,1.0,2.0,NaN,2.0,0.0,0.0,247.0,352.0,125.0,191.0
1502,2017090700,KC,NE,42,27,538.0,425.0,24.0,34.0,38.0,39.0,1.0,2.0,0.0,NaN,0.0,2.0,2.0,352.0,247.0,191.0,125.0


In [112]:
# Creating my target variable wins
combined_df.loc[combined_df['score'] > combined_df['score_against'],'win'] = 1

combined_df['win'].fillna(0,inplace=True)

In [113]:
combined_df.head()

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,avg_kick_dist,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win
0,2015091000,NE,PIT,28,21,366.0,429.0,23.0,25.0,34.0,42.0,1.0,2.0,0.0,39.5,0.0,4.0,2.0,281.0,332.0,82.0,134.0,1.0
1,2015091300,CHI,GB,23,31,416.0,336.0,33.0,28.0,36.0,23.0,1.0,0.0,3.0,37.0,3.0,1.0,1.0,213.0,189.0,189.0,134.0,0.0
2,2015091301,LA,SEA,33,29,354.0,338.0,26.0,32.0,29.0,47.0,3.0,1.0,2.0,29.0,2.0,3.0,3.0,276.0,219.0,76.0,127.0,1.0
3,2015091302,JAX,CAR,9,19,255.0,253.0,21.0,33.0,44.0,33.0,4.0,1.0,2.0,46.5,1.0,4.0,2.0,169.0,158.0,96.0,107.0,0.0
4,2015091303,WAS,MIA,10,16,290.0,264.0,36.0,17.0,32.0,37.0,4.0,2.0,2.0,22.0,1.0,1.0,1.0,188.0,182.0,162.0,75.0,0.0


In [114]:
# Creating additional variables
combined_df['pass_run_ratio'] = round(combined_df['pass_plays'] / combined_df['run_plays'],2)
combined_df['yard_diff'] = combined_df['yards'] - combined_df['yards_against']
combined_df['to_diff'] = combined_df['to_against'] - combined_df['to_for']

combined_df['fg_rate'] = round(combined_df['fg_md'] / combined_df['fg_at'],4)
combined_df['fg_rate_against'] = round(combined_df['fg_md_against'] / combined_df['fg_at_against'],4)

# Setting fg rates to 0% for teams not attempting field goals
combined_df['fg_rate'].fillna(0,inplace=True)
combined_df['fg_rate_against'].fillna(0,inplace=True)

combined_df['pass_yds_per_at'] = combined_df['pass_yards'] / combined_df['pass_plays']
combined_df['run_yds_per_at'] = combined_df['run_yards'] / combined_df['run_plays']

combined_df['pass_yds_per_at_against'] = combined_df['pass_yards_against'] / combined_df['pass_plays_against']
combined_df['run_yds_per_at_against'] = combined_df['run_yards_against'] / combined_df['run_plays_against']

In [115]:
# Team wins overall for sorting
team_wins_overall = []
years = [2015,2016,2017,2018]
year_list = []
team_list = []

for team in teams:
    for year in years:
        home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'H')
                                               ]['home_team_win'])

        away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'A')
                                               ]['away_team_win'])

        # Due to lack of data, normalizing 2018 to 16 week season
        if year == 2018:
            all_wins = np.round((home_wins + away_wins) * 16/14,0)
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)
            
        else:
            all_wins = home_wins + away_wins
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)
    

team_by_wins = pd.DataFrame({'Team':team_list,'wins':team_wins_overall,'year':year_list}).sort_values(by=['wins'])

# Could change this to mathematical bins
bins = [-1, 7, 9, 17]
labels = ['5 or Less', 'Between 6 and 9','10+ Wins']
team_by_wins['binned'] = pd.cut(team_by_wins['wins'], bins=bins, labels=labels)

In [116]:
combined_df['year'] = test_df['game_id'].astype('str').str.extract(r'(^[0-9]{4})').astype('int')

In [117]:
# Adding Win Buckets to combined DF
combined_df = combined_df.merge(team_by_wins[['Team','binned','year']],how='left',on = ['Team','year'])

In [118]:
# Inspecting results
combined_df.head()

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,avg_kick_dist,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win,pass_run_ratio,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,year,binned
0,2015091000,NE,PIT,28,21,366.0,429.0,23.0,25.0,34.0,42.0,1.0,2.0,0.0,39.5,0.0,4.0,2.0,281.0,332.0,82.0,134.0,1.0,1.48,-63.0,1.0,0.0,0.5,8.264706,3.565217,7.904762,5.360000,2015,10+ Wins
1,2015091300,CHI,GB,23,31,416.0,336.0,33.0,28.0,36.0,23.0,1.0,0.0,3.0,37.0,3.0,1.0,1.0,213.0,189.0,189.0,134.0,0.0,1.09,80.0,-1.0,1.0,1.0,5.916667,5.727273,8.217391,4.785714,2015,5 or Less
2,2015091301,LA,SEA,33,29,354.0,338.0,26.0,32.0,29.0,47.0,3.0,1.0,2.0,29.0,2.0,3.0,3.0,276.0,219.0,76.0,127.0,1.0,1.12,16.0,-2.0,1.0,1.0,9.517241,2.923077,4.659574,3.968750,2015,5 or Less
3,2015091302,JAX,CAR,9,19,255.0,253.0,21.0,33.0,44.0,33.0,4.0,1.0,2.0,46.5,1.0,4.0,2.0,169.0,158.0,96.0,107.0,0.0,2.10,2.0,-3.0,0.5,0.5,3.840909,4.571429,4.787879,3.242424,2015,5 or Less
4,2015091303,WAS,MIA,10,16,290.0,264.0,36.0,17.0,32.0,37.0,4.0,2.0,2.0,22.0,1.0,1.0,1.0,188.0,182.0,162.0,75.0,0.0,0.89,26.0,-2.0,0.5,1.0,5.875000,4.500000,4.918919,4.411765,2015,Between 6 and 9


In [86]:
combined_df.fg_at.sum()

3898.0

In [87]:
combined_df.fg_md.sum()

3296.0

In [119]:
# Saving end of game stats to csv
combined_df.to_csv('nfl_end_of_game.csv')